In [1]:
import pandas as pd
import pickle
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import os

In [2]:
def table_parser(page):
    file = open(page)
    # page = page.split('\n')
    table = []
    num = 0
    for line in file:
        if 'Grade' in line:
            num += 1
        if num > 0 and len(line.split())!=0:
            num += 1
        if 3 <= num < 21:
            # line = line.rstrip()
            if len(line.split())!=0:
                split_line = line.split(' ')
                split_line = [x for x in split_line if x != '']
                strip_line = split_line[:16]
                table.append(strip_line)
    df = pd.DataFrame(table)
    df.drop([0,1,2,3,4,6,7,8,9,11,12,13,14],axis=1,inplace=True)
    df.drop([15,16,17],inplace=True)
    df.columns = ['WG', 'WL', 'WS']
    df.index = range(1,16)
    
    file.close()
    return df

In [3]:
list(table_parser('docs/AK/007,007,AK,Ancorage/09Sep1997/AK-007,007,AK,Ancorage-09Sep1997.html')['WG'])

['14.78',
 '15.94',
 '17.10',
 '18.26',
 '19.42',
 '20.57',
 '21.73',
 '22.89',
 '24.05',
 '25.21',
 '26.36',
 '27.53',
 '28.68',
 '29.84',
 '31.00']

In [78]:
test = pd.DataFrame(test_df)

In [13]:
dic = {}
for state in os.listdir('docs'):
    dic[state] = {}
    for county in os.listdir('docs/'+state):
        
        name = county.split(',')[3]
        dic[state][name] = {}
        for date in os.listdir('docs/'+state+'/'+county):
            try:
                dic[state][name][date] = {}
                temp = table_parser('docs/{}/{}/{}/{}-{}-{}.html'.format(state, county, date,state, county, date))
                dic[state][name][date]['WG'] = temp['WG']
                dic[state][name][date]['WL'] = temp['WL']
                dic[state][name][date]['WS'] = temp['WS']
            except ValueError:
                print(state, county, date)

AZ 009,009,RUS,Gila County 07May1996
AZ 009,009,RUS,Yavapai County 07May1996
AZ 010,010,RUS,Graham County 07May1996
AZ 010,010,RUS,Greenlee County 07May1996


In [117]:
# dic = {'A': {'a': {1: [2,3,4,5,6],
#    ...:                           2: [2,3,4,5,6]},
#    ...:                     'b': {1: [2,3,4,5,6],
#    ...:                           2: [2,3,4,5,6]}},
#    ...:               'B': {'a': {1: [2,3,4,5,6],
#    ...:                           2: [2,3,4,5,6]},
#    ...:                     'b': {1: [2,3,4,5,6],
#    ...:                           2: [2,3,4,5,6]}}}

In [14]:
reform = {(level1_key, level2_key, level3_key): values
           for level1_key, level2_dict in dic.items()
           for level2_key, level3_dict in level2_dict.items()
           for level3_key, level4_dict in level3_dict.items()
#            for level4_key, level5_dict in level4_dict.items()
           for level4_key, values      in level4_dict.items()}

In [21]:
test = pd.DataFrame(reform).T
test.head()

1      2      3      4      5      6      7      8   \
AK Ancorage 09Sep1997  22.34  23.50  24.66  25.82  26.98  28.13  29.29  30.45   
            09Sep2008  25.59  27.47  29.37  31.26  33.18  35.05  36.95  38.85   
            10Sep1996  22.22  23.35  24.48  25.60  26.73  27.87  29.00  30.12   
            11Sep2007  24.84  26.67  28.51  30.35  32.21  34.03  35.87  37.72   
            12Sep2006  24.39  26.19  28.00  29.81  31.63  33.42  35.23  37.04   

                          9      10     11     12     13     14     15  
AK Ancorage 09Sep1997  31.61  32.77  33.29  33.96  34.80  35.80  36.95  
            09Sep2008  40.75  42.64  43.53  44.64  46.03  47.71  49.63  
            10Sep1996  31.25  32.38  32.87  33.50  34.29  35.24  36.32  
            11Sep2007  39.56  41.40  42.26  43.34  44.69  46.32  48.18  
            12Sep2006  38.85  40.66  41.50  42.56  43.89  45.49  47.32

In [16]:
reform

{('AK', 'Ancorage', '09Sep1997'): 1     22.34
 2     23.50
 3     24.66
 4     25.82
 5     26.98
 6     28.13
 7     29.29
 8     30.45
 9     31.61
 10    32.77
 11    33.29
 12    33.96
 13    34.80
 14    35.80
 15    36.95
 Name: WS, dtype: object, ('AK', 'Ancorage', '09Sep2008'): 1     25.59
 2     27.47
 3     29.37
 4     31.26
 5     33.18
 6     35.05
 7     36.95
 8     38.85
 9     40.75
 10    42.64
 11    43.53
 12    44.64
 13    46.03
 14    47.71
 15    49.63
 Name: WS, dtype: object, ('AK', 'Ancorage', '10Sep1996'): 1     22.22
 2     23.35
 3     24.48
 4     25.60
 5     26.73
 6     27.87
 7     29.00
 8     30.12
 9     31.25
 10    32.38
 11    32.87
 12    33.50
 13    34.29
 14    35.24
 15    36.32
 Name: WS, dtype: object, ('AK', 'Ancorage', '11Sep2007'): 1     24.84
 2     26.67
 3     28.51
 4     30.35
 5     32.21
 6     34.03
 7     35.87
 8     37.72
 9     39.56
 10    41.40
 11    42.26
 12    43.34
 13    44.69
 14    46.32
 15    48.18
 Name: WS, dt